In [1]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import json
import re
from requests.auth import HTTPBasicAuth
from pandas import json_normalize

In [5]:
listing_csv = pd.read_csv('../data/no_dup_listing.csv')
listing_csv

,Unnamed: 0,index,Unnamed: 0.1,listing_id,user_id,shop_id,title,description,state,creation_timestamp,...,has_variations,should_auto_renew,language,taxonomy_id,production_partners,skus,price_fixed,price.amount,price.divisor,price.currency_code
0,12116,0,0,740223621,19840612,6766412,If You Want to Change the World Go Home and Lo...,"“If you want to change the world, go home and ...",active,10/28/2020 4:51,...,False,False,en-US,2078,[],[],$19.99,1999,100,USD
1,12117,1,1,490998782,38954524,8718556,"Digitized Art (not a mug): Love, For God so lo...","IMPORTANT NOTE: THIS IS NOT A MUG FOR SALE, B...",active,10/28/2020 16:12,...,False,True,en-US,119,[],[],$4.00,400,100,USD
2,12118,2,2,518644764,101112796,14333509,They Broke Bread Sign - Acts 2:46 - Wood Sign ...,~BROKE BREAD SIGN~ Wooden Decorative Inspirati...,active,10/28/2020 16:34,...,True,True,en-US,2844,[],[],$105.00,10500,100,USD
3,12119,3,3,644931979,162226534,18110774,Scripture Wall Art Bible Verse Wall Art Bible ...,Psalm 46:10 Bible Quote Print Bible Verses Pri...,active,10/28/2020 19:41,...,False,True,en-US,2078,[],[],$6.99,699,100,CAD
4,12120,4,4,581224258,121658751,15800102,Hand Lettered Word of the Year - Service - IN...,Hand Lettered Word of the Year - Service - INS...,active,10/28/2020 23:40,...,False,True,en-US,2078,[],[],$5.00,500,100,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24436,24436,1967,1967,824624370,278682773,22430746,"Hand Lettered Vellum Place Card with Ribbon, W...",***D E S C R I P T I O N***\nThis listing is f...,active,6/21/2022 18:36,...,True,False,en-US,6056,[],[],$2.39,239,100,USD
24437,24437,1968,1968,490919268,14899819,6363955,Round Return Address Stamp Personalized Self I...,Round Return Address Stamp Personalized Circle...,active,6/21/2022 18:47,...,True,False,en-US,6365,[],[],$32.00,3200,100,USD
24438,24438,1969,1969,1079434119,5952033,5311677,Kimberly Beach Wedding Invitation Sample with ...,NEW Kimberly Watercolor Wedding Invitation Sam...,active,6/21/2022 18:59,...,True,True,en-US,1676,[],[],$5.00,500,100,USD
24439,24439,1970,1970,221698849,59805461,10514292,PHYSICAL CUSTOM PRINT Hand Lettered Watercolor...,Watercolor calligraphy prints each hand letter...,active,6/21/2022 19:02,...,True,False,en-US,1676,[],[],$12.00,1200,100,USD


In [7]:
above_100rv_csv = pd.read_csv('../data/above_100.csv')
above_100rv_csv

,Unnamed: 0,shop_id,shop_name,url,create_date,create_date.1,num_favorers,transaction_sold_count,review_average,review_count
0,321,5116051,sugarloop,https://www.etsy.com/shop/sugarloop,2005-09-23 08:54:52,2005-09-23 08:54:52,19426,53985,4.9910,1111.0
1,3447,5136616,CAprintables,https://www.etsy.com/shop/CAprintables,2006-09-06 04:00:18,2006-09-06 04:00:18,1016,6151,4.9529,191.0
2,843,5147375,CupcakesAndStickies,https://www.etsy.com/shop/CupcakesAndStickies,2007-01-05 00:50:37,2007-01-05 00:50:37,2627,13421,4.9677,186.0
3,1746,5157128,SouthPacific,https://www.etsy.com/shop/SouthPacific,2007-03-06 04:00:26,2007-03-06 04:00:26,7855,41238,4.9369,396.0
4,515,5157400,LizzyClara,https://www.etsy.com/shop/LizzyClara,2007-03-07 22:24:25,2007-03-07 22:24:25,3358,7310,4.9657,321.0
...,...,...,...,...,...,...,...,...,...,...
801,1901,32433825,PoppyAndMagnoliaCo,https://www.etsy.com/shop/PoppyAndMagnoliaCo,2021-11-04 03:49:56,2021-11-04 03:49:56,126,1022,4.9948,191.0
802,1419,33237074,MulberryStPaperGoods,https://www.etsy.com/shop/MulberryStPaperGoods,2021-12-08 02:05:41,2021-12-08 02:05:41,203,9370,4.8936,592.0
803,3268,34399588,EunoiaCraftUS,https://www.etsy.com/shop/EunoiaCraftUS,2022-02-12 20:37:12,2022-02-12 20:37:12,81,4667,4.7244,312.0
804,3597,34669814,LemonSvgDesign,https://www.etsy.com/shop/LemonSvgDesign,2022-03-02 16:30:41,2022-03-02 16:30:41,48,1595,4.9153,118.0


In [8]:
shop_id = listing_csv['shop_id']

In [11]:
combo_filtered = listing_csv[shop_id.isin(above_100rv_csv['shop_id'])]

In [14]:
combo_filtered.reset_index(drop=True)

,Unnamed: 0,index,Unnamed: 0.1,listing_id,user_id,shop_id,title,description,state,creation_timestamp,...,has_variations,should_auto_renew,language,taxonomy_id,production_partners,skus,price_fixed,price.amount,price.divisor,price.currency_code
0,3,3,3,717730423,43040627,9039454,Custom Name Wooden Plank Laser Cut Sign - Lett...,| ITEM DETAILS |\nCustom Wooden Name Plank\nPi...,active,10/29/2020 15:21,...,True,True,en-US,1334,[],[],$75.00,7500,100,USD
1,7,7,7,535429603,23654923,7148172,Children&#39;s Crocodile Birthday Card - Circu...,"A simple, bold and fun hand lettered child&#39...",active,10/30/2020 15:21,...,True,True,en-US,1264,[],[],$3.25,325,100,GBP
2,10,10,10,717727393,43040627,9039454,Custom Wreath Name Sign - 20 inch or 24 inch -...,**FREE SHIPPING for the LARGE ITEMS IS ONLY AV...,active,10/31/2020 3:00,...,True,True,en-US,6121,[],[],$112.00,11200,100,USD
3,11,11,11,639925062,61609693,15355183,"Christmas Gift Tag Stickers, Christmas Rose Go...","A sheet of real foil Christmas gift tags, perf...",active,10/31/2020 10:40,...,False,True,en-US,1322,[],[],$3.40,340,100,EUR
4,12,12,12,673253704,28608137,7631121,Absolute Best Mum card,"&quot;Mum, You&#39;re the Absolute Best&quot;\...",active,10/31/2020 18:13,...,False,True,en-US,1281,[],[],$3.00,300,100,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8308,24436,1967,1967,824624370,278682773,22430746,"Hand Lettered Vellum Place Card with Ribbon, W...",***D E S C R I P T I O N***\nThis listing is f...,active,6/21/2022 18:36,...,True,False,en-US,6056,[],[],$2.39,239,100,USD
8309,24437,1968,1968,490919268,14899819,6363955,Round Return Address Stamp Personalized Self I...,Round Return Address Stamp Personalized Circle...,active,6/21/2022 18:47,...,True,False,en-US,6365,[],[],$32.00,3200,100,USD
8310,24438,1969,1969,1079434119,5952033,5311677,Kimberly Beach Wedding Invitation Sample with ...,NEW Kimberly Watercolor Wedding Invitation Sam...,active,6/21/2022 18:59,...,True,True,en-US,1676,[],[],$5.00,500,100,USD
8311,24439,1970,1970,221698849,59805461,10514292,PHYSICAL CUSTOM PRINT Hand Lettered Watercolor...,Watercolor calligraphy prints each hand letter...,active,6/21/2022 19:02,...,True,False,en-US,1676,[],[],$12.00,1200,100,USD


In [18]:
combo_filtered.sort_values(by=['creation_timestamp'], ascending = False)

,Unnamed: 0,index,Unnamed: 0.1,listing_id,user_id,shop_id,title,description,state,creation_timestamp,...,has_variations,should_auto_renew,language,taxonomy_id,production_partners,skus,price_fixed,price.amount,price.divisor,price.currency_code
108,40,40,40,461269366,43040627,9039454,Birthday Girl Laser Cut Gold Cake Topper - Han...,"This cake topper is hand lettered, digitized, ...",active,7/31/2021 9:20,...,True,True,en-US,1334,[],[],$25.00,2500,100,USD
100,12180,65,65,776056532,26909963,7517236,Flawed and still worthy,This sign is hand lettered and painted to orde...,active,7/31/2021 7:30,...,True,True,en-US,2844,[],[],$35.00,3500,100,USD
99,12179,64,64,865503479,26909963,7517236,As You Wish - Buttercups,This sign is hand lettered and painted to orde...,active,7/31/2021 7:30,...,True,True,en-US,2844,[],[],$35.00,3500,100,USD
96,35,35,35,619014883,23654923,7148172,Hard Times True Friends Small Card - Friendshi...,This hand-lettered friendship greeting card is...,active,7/31/2021 7:21,...,True,True,en-US,1268,[],[],$2.50,250,100,GBP
95,12176,61,61,891215102,26909963,7517236,Something Tells me I’m going to love you forev...,This sign is hand lettered and painted to orde...,active,7/31/2021 7:21,...,True,True,en-US,2844,[],[],$35.00,3500,100,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,3,3,3,717730423,43040627,9039454,Custom Name Wooden Plank Laser Cut Sign - Lett...,| ITEM DETAILS |\nCustom Wooden Name Plank\nPi...,active,10/29/2020 15:21,...,True,True,en-US,1334,[],[],$75.00,7500,100,USD
113,42,42,42,291145415,43040627,9039454,100 Crest Laser Cut Cake Topper for 100th Day ...,100 Crest Cake Topper | Pictured: GOLD\n\nThis...,active,10/28/2021 4:31,...,True,True,en-US,1331,[],[],$25.00,2500,100,USD
124,45,45,45,584087059,100636681,14008975,Printable OR Printed Picture Thank You Postcar...,Printable OR Printed Photo Thank You Postcards...,active,10/28/2021 23:32,...,True,False,en-US,1679,[],[],$18.48,1848,100,USD
121,12192,77,77,1026877734,173767613,18449821,He is the Mountain Mover Hand-Lettered Christi...,★ Sizes available:\n - A5: 210 x 148 mm /...,active,10/28/2021 22:16,...,True,True,en-US,1029,[],[],$8.00,800,100,GBP
